In [1]:
import pyodbc
from import_function import import_scoring_data, import_data
from features_by_customer_type import customer_type_features
conn = pyodbc.connect(dsn='VerticaProd')
x=120
ct='New'
response=['ABANDONED']

cont_features, bool_features, catag_features, cont_score_features, bool_score_features, catag_score_features=customer_type_features(ct)
scoring_data = "SELECT A.* FROM (SELECT SUP_REMIC_TAXABLE_INCOME,	SUP_REMIC_SCHQ_INCOME,	SUP_REMIC_EXCESS_INCLUSION,	SUP_RE_TOTAL_INCOME,	SUP_RE_PROFESSIONAL_INCOME,	SUP_RE_PERSONAL_DAYS,	SUP_RE_INCOME_ROYALTIES,	SUP_RE_INCOME_RENTS,	SUP_RE_EXPENSES_UTILITIES,	SUP_RE_EXPENSES_TRAVEL,	SUP_RE_EXPENSES_TOTAL,	SUP_RE_EXPENSES_TAXES,	SUP_RE_EXPENSES_SUPPLIES,	SUP_RE_EXPENSES_REPAIRS,	SUP_RE_EXPENSES_OTHER_INTEREST,	SUP_RE_EXPENSES_MORTGAGE_INTEREST,	SUP_RE_EXPENSES_MANAGEMENT,	SUP_RE_EXPENSES_LEGAL,	SUP_RE_EXPENSES_INSURANCE,	SUP_RE_EXPENSES_DEPRECIATION,	SUP_RE_EXPENSES_DEDUCTIBLE_LOSS,	SUP_RE_EXPENSES_COMMISSIONS,	SUP_RE_EXPENSES_CLEANING,	SUP_RE_EXPENSES_ADVERTISING,	SUP_PS_TOTAL_INCOME,	SUP_PS_SEC179_EXPENSE_DEDUCTION,	SUP_PS_PY_LOSS,	SUP_PS_PASSIVE_LOSS,	SUP_PS_PASSIVE_INCOME,	SUP_PS_NONPASSIVE_LOSS,	SUP_PS_NONPASSIVE_INCOME,	SUP_PS_AT_RISK,	SUP_FARM_RENTAL_INCOME,	SUP_FARM_GROSS_INCOME,	SUP_EST_TOTAL_INCOME,	SUP_EST_PASSIVE_LOSS,	SUP_EST_PASSIVE_INCOME,	SUP_EST_NONPASSIVE_LOSS,	SUP_EST_NONPASSIVE_INCOME,	NUM_EXEMPTIONS,	NUM_DEPENDENTS,	FLAG_OLD_OR_BLIND,	BUS_WILL_FILE_1099,	BUS_VEHICLE_MILES_OTHER,	BUS_VEHICLE_MILES_COMMUTE,	BUS_VEHICLE_MILES_BUSINESS,	BUS_STATUTORY_EMPLOYEE,	BUS_START_ACQUIRE,	BUS_REQUIRE_1099,	BUS_OTHER_INCOME,	BUS_NET_PROFIT,	BUS_MATERIAL_PARTICIPATE,	BUS_INVESTMENT_AT_RISK,	BUS_GROSS_PROFIT,	BUS_GROSS_INCOME,	BUS_EXPENSE_WAGES,	BUS_EXPENSE_UTILITIES,	BUS_EXPENSE_USE_OF_HOME,	BUS_EXPENSE_TRAVEL,	BUS_EXPENSE_TOTAL,	BUS_EXPENSE_TAXES,	BUS_EXPENSE_SUPPLIES,	BUS_EXPENSE_REPAIRS,	BUS_EXPENSE_RENT_VEHICLES,	BUS_EXPENSE_RENT_OTHER,	BUS_EXPENSE_PROFIT_TENTATIVE,	BUS_EXPENSE_PENSION,	BUS_EXPENSE_OTHER_INTEREST,	BUS_EXPENSE_OTHER,	BUS_EXPENSE_OFFICE,	BUS_EXPENSE_MORTGAGE_INTEREST,	BUS_EXPENSE_MEALS,	BUS_EXPENSE_LEGAL,	BUS_EXPENSE_INSURANCE,	BUS_EXPENSE_EMPLOYEE_BENEFITS,	BUS_EXPENSE_DEPRECIATION,	BUS_EXPENSE_DEPLETION,	BUS_EXPENSE_CONTRACT_LABOR,	BUS_EXPENSE_COMMISSIONS,	BUS_EXPENSE_CAR,	BUS_EXPENSE_ADVERTISING,	BUS_COGS_TOTAL,	BUS_COGS_SUPPLIES,	BUS_COGS_PURCHASES,	BUS_COGS_LABOR,	BUS_COGS_INVENTORY_START,	BUS_COGS_INVENTORY_END,	BUS_COGS,	AMOUNT_UNREPORTED_SS_MEDICARE_TAX,	AMOUNT_UNEMPLOYMENT,	AMOUNT_TUITION,	AMOUNT_TOTAL_TAX,	AMOUNT_TOTAL_PAYMENTS,	AMOUNT_TOTAL_INTEREST_PAID,	AMOUNT_TOTAL_INCOME,	AMOUNT_TOTAL_DEDUCTIONS,	AMOUNT_TOTAL_DEDUCTIBLE_EXPENSES,	AMOUNT_TOTAL_CREDITS,	AMOUNT_TAXES_PAID,	AMOUNT_TAXABLE_SOCIAL_SEC,	AMOUNT_TAXABLE_OFFSETS,	AMOUNT_TAXABLE_IRA,	AMOUNT_TAXABLE_INTEREST,	AMOUNT_TAXABLE_INCOME,	AMOUNT_TAX_PREP_FEES,	AMOUNT_TAX_DUE,	AMOUNT_TAX_CREDITS,	AMOUNT_TAX,	AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION,	AMOUNT_STATE_LOCAL_TAX,	AMOUNT_STATE_LOCAL_SALES_TAX,	AMOUNT_SOCIAL_SEC,	AMOUNT_SELF_EMPLOYMENT_TAX,	AMOUNT_SELF_EMPLOYMENT_RETIREMENT,	AMOUNT_SELF_EMPLOYMENT_HEALTH_INSURANCE,	AMOUNT_SCHE,	AMOUNT_SALARIES_AND_WAGES,	AMOUNT_RETIREMENT_SAVINGS_CREDIT,	AMOUNT_RESIDENTIAL_ENERGY_CREDIT,	AMOUNT_REFUND,	AMOUNT_REAL_ESTATE_TAX,	AMOUNT_QUALIFIED_DIVIDENDS,	AMOUNT_PERSONAL_PROPERTY_TAXES,	AMOUNT_PAID_WITH_EXTENSION,	AMOUNT_OTHER_TAXES,	AMOUNT_OTHER_PAYMENTS,	AMOUNT_OTHER_INCOME,	AMOUNT_OTHER_GAIN,	AMOUNT_OTHER_DEDUCTIBLE_TAXES,	AMOUNT_OTHER_DEDUCTIBLE_EXPENSES,	AMOUNT_OTHER_CREDITS,	AMOUNT_ORDINARY_DIVIDENDS,	AMOUNT_NT_COMBAT_PAY,	AMOUNT_MOVING_EXPENSE,	AMOUNT_MORTGAGE_POINTS_NON_1098,	AMOUNT_MORTGAGE_INTEREST_NON_1098,	AMOUNT_MORTGAGE_INTEREST,	AMOUNT_MORTGAGE_INSURANCE,	AMOUNT_MISC_DEDUCTIONS,	AMOUNT_MEDICAL_DENTAL_EXPENSES_DEDUCTION,	AMOUNT_MEDICAL_DENTAL_EXPENSES,	AMOUNT_IRA_DISTRIBUTIONS,	AMOUNT_IRA_DEDUCTION,	AMOUNT_INVESTMENT_INTEREST_PAID,	AMOUNT_INCOME_TAX_WITHHELD,	AMOUNT_INCOME_TAX,	AMOUNT_HSA,	AMOUNT_HOPE_CREDIT,	AMOUNT_HOMEBUYER_CREDIT_REPAYMENT,	AMOUNT_FUEL_TAX_CREDIT,	AMOUNT_FOREIGN_TAX_CREDIT,	AMOUNT_FARM_INCOME,	AMOUNT_EXPENSES_DEDUCTION,	AMOUNT_EXEMPTIONS,	AMOUNT_EXCESS_SS_RRTA_WITHHELD,	AMOUNT_ESTIMATED_TAX_PENALTY,	AMOUNT_ESTIMATED_TAX,	AMOUNT_EMPLOYEE_EXPENSES,	AMOUNT_EITC,	AMOUNT_EDUCATION_CREDIT,	AMOUNT_EARLY_WITHDRAWAL_PENALTY,	AMOUNT_DOMESTIC_PRODUCTION_DEDUCTION,	AMOUNT_DISABLED_CREDIT,	AMOUNT_DEDUCTIBLE_SELF_EMPLOYMENT_TAX,	AMOUNT_CHILD_CREDIT,	AMOUNT_CHILD_CARE_CREDIT,	AMOUNT_CHARITABLE_CONTRIBUTIONS_NONCASH,	AMOUNT_CHARITABLE_CONTRIBUTIONS_CASH,	AMOUNT_CHARITABLE_CONTRIBUTIONS_CARRYOVER,	AMOUNT_CHARITABLE_CONTRIBUTIONS,	AMOUNT_CERTAIN_BUSINESS_EXPENSE,	AMOUNT_CASUALTY_LOSSES,	AMOUNT_CAPITAL_GAIN,	AMOUNT_BUSINESS_INCOME,	AMOUNT_AMT,	AMOUNT_ALIMONY_PAID,	AMOUNT_ALIMONY_INCOME,	AMOUNT_ADJUSTMENTS,	AGI,	AGE_TAXPAYER,	AGE_SPOUSE,	AGE_DEPENDENT_MIN,	AGE_DEPENDENT_MAX,	AGE_DEPENDENT_AVG,	CUSTOMER_KEY,	ABANDONED,	TTO_FLAG,	STATE_ATTACH_COUNT,	MINDBENDER_FLAG,	CA_REFUND_TRANSFER_FLAG,	NON_CA_REFUND_TRANSFER_FLAG,	REFUND_TRANSFER_FLAG,	CA_AUDIT_DEFENSE_FLAG,	NON_CA_AUDIT_DEFENSE_FLAG,	AUDIT_DEFENSE_FLAG,	CA_MAX_FLAG,	NON_CA_MAX_FLAG,	MAX_FLAG,	PS_FLAG,	PRS_SCORE,	REJECT_COUNT,	TAX_WEEK,	TAX_DAY,	CORE_FLAG,	REQUIRED_TAKE_FLAG,	FSCHC_FLAG,	FSCHCEZ_FLAG,	FSCHE_FLAG,	FSCHA_FLAG,	FSCHD_FLAG,	FSCHF_FLAG,	MISC1099_FLAG,	ACCEPTED_EFILE,	NUM_CARE_CONTACTS,	RT_FLAG,	PRE_UPVOTES,	PRE_DOWNVOTES,	PRE_TOTAL_VOTES,	POST_UPVOTES,	POST_DOWNVOTES,	POST_TOTAL_VOTES,	SUP_RE_WILL_FILE_1099,	SUP_RE_TYPE_SINGLE_FAMILY,	SUP_RE_TYPE_SHORT_RENTAL,	SUP_RE_TYPE_SELF_RENTAL,	SUP_RE_TYPE_ROYALTIES,	SUP_RE_TYPE_OTHER,	SUP_RE_TYPE_MULTI_FAMILY,	SUP_RE_TYPE_LAND,	SUP_RE_TYPE_COMMERCIAL,	SUP_RE_REQUIRE_1099,	SUP_PS_SCORP,	SUP_PS_PARTNERSHIP_FOREIGN,	SUP_PS_PARTNERSHIP,	NUM_W2,	NUM_SCHE,	NUM_SCHC,	FLAG_ITEMIZED_DEDUCTIONS,	BUS_VEHICLE_PERSONAL_OFFDUTY,	BUS_VEHICLE_PERSONAL_ANOTHER,	BUS_COGS_METHOD_OTHER,	BUS_COGS_METHOD_COST_OR_MARKET,	BUS_COGS_METHOD_COST,	BUS_ACCOUNTING_METHOD_OTHER,	BUS_ACCOUNTING_METHOD_CASH,	BUS_ACCOUNTING_METHOD_ACCRUAL,	student_taxpayer,	_taxpayer,	retired_taxpayer,	unemployed_taxpayer,	cashier_taxpayer,	teacher_taxpayer,	manager_taxpayer,	sales_taxpayer,	customerservice_taxpayer,	laborer_taxpayer,	cook_taxpayer,	server_taxpayer,	retail_taxpayer,	engineer_taxpayer,	military_taxpayer,	cna_taxpayer,	salesassociate_taxpayer,	registerednurse_taxpayer,	truckdriver_taxpayer,	driver_taxpayer,	mechanic_taxpayer,	nurse_taxpayer,	waitress_taxpayer,	supervisor_taxpayer,	labor_taxpayer,	construction_taxpayer,	warehouse_taxpayer,	administrativeassistant_taxpayer,	accountant_taxpayer,	customerservicerep_taxpayer,	clerk_taxpayer,	receptionist_taxpayer,	medicalassistant_taxpayer,	disabled_taxpayer,	softwareengineer_taxpayer,	assistantmanager_taxpayer,	officemanager_taxpayer,	electrician_taxpayer,	technician_taxpayer,	none_taxpayer,	machineoperator_taxpayer,	caregiver_taxpayer,	employed_taxpayer,	bartender_taxpayer,	projectmanager_taxpayer,	maintenance_taxpayer,	welder_taxpayer,	housekeeper_taxpayer,	homemaker_taxpayer,	foodservice_taxpayer,	selfemployed_taxpayer,	chef_taxpayer,	csr_taxpayer,	warehouseworker_taxpayer,	factoryworker_taxpayer,	attorney_taxpayer,	consultant_taxpayer,	policeofficer_taxpayer,	carpenter_taxpayer,	secretary_taxpayer,	rn_taxpayer,	banker_taxpayer,	operator_taxpayer,	worker_taxpayer,	security_taxpayer,	production_taxpayer,	socialworker_taxpayer,	securityofficer_taxpayer,	pharmacytechnician_taxpayer,	machinist_taxpayer,	barista_taxpayer,	analyst_taxpayer,	deliverydriver_taxpayer,	stocker_taxpayer,	factory_taxpayer,	dentalassistant_taxpayer,	custodian_taxpayer,	generalmanager_taxpayer,	employee_taxpayer,	salesmanager_taxpayer,	retailmanager_taxpayer,	housekeeping_taxpayer,	storemanager_taxpayer,	securityguard_taxpayer,	na_taxpayer,	painter_taxpayer,	accountmanager_taxpayer,	correctionalofficer_taxpayer,	lpn_taxpayer,	operationsmanager_taxpayer,	bankteller_taxpayer,	fastfood_taxpayer,	manufacturing_taxpayer,	crewmember_taxpayer,	generallabor_taxpayer,	plumber_taxpayer,	marketing_taxpayer,	dispatcher_taxpayer,	hostess_taxpayer,	management_taxpayer,	janitor_taxpayer,	homemaker_spouse,	retired_spouse,	unemployed_spouse,	teacher_spouse,	student_spouse,	housewife_spouse,	none_spouse,	disabled_spouse,	home_maker_spouse,	house_wife_spouse,	stayathomemom_spouse,	nurse_spouse,	manager_spouse,	registerednurse_spouse,	sales_spouse,	customerservice_spouse,	cashier_spouse,	na_spouse,	laborer_spouse,	retail_spouse,	officemanager_spouse,	administrativeassistant_spouse,	engineer_spouse,	secretary_spouse,	accountant_spouse,	cook_spouse,	receptionist_spouse,	truckdriver_spouse,	self_employed_spouse,	mechanic_spouse,	rn_spouse,	server_spouse,	cna_spouse,	driver_spouse,	supervisor_spouse,	medicalassistant_spouse,	clerk_spouse,	socialworker_spouse,	projectmanager_spouse,	salesassociate_spouse,	mother_spouse,	waitress_spouse,	military_spouse,	construction_spouse,	attorney_spouse,	electrician_spouse,	dentalassistant_spouse,	stayathomemother_spouse,	customerservicerep_spouse,	mom_spouse,	substituteteacher_spouse,	consultant_spouse,	maintenance_spouse,	policeofficer_spouse,	banker_spouse,	labor_spouse,	welder_spouse,	softwareengineer_spouse,	na_spouse2,	technician_spouse,	marketing_spouse,	assistantmanager_spouse,	bankteller_spouse,	stayathomeparent_spouse,	caregiver_spouse,	warehouse_spouse,	educator_spouse,	carpenter_spouse,	bookkeeper_spouse,	housekeeper_spouse,	physician_spouse,	clerical_spouse,	analyst_spouse,	paralegal_spouse,	pharmacist_spouse,	machineoperator_spouse,	chef_spouse,	physicaltherapist_spouse,	humanresources_spouse,	lpn_spouse,	foodservice_spouse,	pharmacytechnician_spouse,	executiveassistant_spouse,	officeassistant_spouse,	disable_spouse,	salesmanager_spouse,	accountmanager_spouse,	professor_spouse,	hairstylist_spouse,	machinist_spouse,	selfemployed_spouse,	custodian_spouse,	factoryworker_spouse,	retailmanager_spouse,	director_spouse,	graphicdesigner_spouse,	bartender_spouse,	administrator_spouse,	home_spouse,	accounting_spouse,	unemployeed_spouse,	ORDER_AMOUNT,	FEDERAL_REVENUE,	MINDBENDER_REVENUE,	CA_REFUND_TRANSFER_REVENUE,	NON_CA_REFUND_TRANSFER_REVENUE,	REFUND_TRANSFER_REVENUE,	CA_AUDIT_DEFENSE_REVENUE,	NON_CA_AUDIT_DEFENSE_REVENUE,	AUDIT_DEFENSE_REVENUE,	CA_MAX_REVENUE,	NON_CA_MAX_REVENUE,	MAX_REVENUE,	PS_REVENUE,	FLAG_ITEMIZE_SEPARATELY,	TOTAL_REVENUE,	STATE_REVENUE,	START_SKU,	ENTRY_PAGE_GROUP,	CHANNEL,	IMPORT_TYPE,	START_SKU_ROLLUP,	FED_FORM_TYPE,	PRODUCT_ROLLUP,	PRODUCT_EDITION_DESCRIPTION,	CUSTOMER_TYPE_ROLLUP,	CUSTOMER_TYPE,	CUSTOMER_DEFINITION,	CUSTOMER_DEFINITION_ADJ,	NEW_CUSTOMER_DEFINITION,	COMPLETED_SKU,	LAST_STATUS,	FILING_STATUS,	DMA_AREA FROM  CTG_ANALYTICS_WS.SM_RETENTION_MODEL  WHERE   TAX_YEAR=2016 AND CUSTOMER_DEFINITION_ADJ='NEW TO TURBOTAX' )A inner join (SELECT DISTINCT CUSTOMER_KEY FROM  CTG_ANALYTICS_WS.SM_RETENTION_MODEL  WHERE   TAX_YEAR=2016 AND TAX_DAY=120) B ON A.CUSTOMER_KEY=B.CUSTOMER_KEY limit 100"
data = "SELECT * FROM (SELECT * FROM  CTG_ANALYTICS_WS.SM_RETENTION_MODEL  WHERE   TAX_YEAR=2015 AND  CUSTOMER_DEFINITION_ADJ IN ('NEW TO TURBOTAX')    )A   order by random() limit 1000"
df_no_pca = import_data(data, cont_features, bool_features,response, catag_features)
scoring_df=import_scoring_data(scoring_data, cont_score_features, bool_score_features,catag_score_features)





c:\python27\lib\site-packages\pandas\core\frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


df_cont done
data_scaled done
just_dummies done
df_trans done
new feature list done


import_function.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_bool.drop([f], axis=1, inplace=True)


In [2]:
from bin_cont_features import bin_pca
j='True'
response='ABANDONED'
df_pca=df_no_pca
dummy_pca, length_dict = bin_pca(df_pca,response, j)
print list(dummy_pca), dummy_pca.head()
print dict

['<=1.5', '<=0.5', 'between 0.5 and 1.5', '>1.5', 'between 1.5 and 3.5', '>=3.5']
['<=0.5', '>0.5', 'between 0.5 and 2.5', '>=2.5']
['<=3307.0', '<=759.0', 'between 759.0 and 3307.0', '>3307.0']
['<=13217.5', '<=9206.5', 'between 9206.5 and 13217.5', '>13217.5', 'between 13217.5 and 18428.5', '>=18428.5']
['<=107.5', '<=87.5', 'between 87.5 and 107.5', '>107.5', 'between 107.5 and 14577.0', '>=14577.0']
['<=823.0', '>823.0', 'between 823.0 and 6052.0', '>=6052.0']
['<=2516.5', '<=617.0', 'between 617.0 and 2516.5', '>2516.5', 'between 2516.5 and 11738.0', '>=11738.0']
['<=525.0', '>525.0', 'between 525.0 and 13198.0', '>=13198.0']
['<=241.5', '>241.5', 'between 241.5 and 3580.0', '>=3580.0']
['<=279.0', '<=55.5', 'between 55.5 and 279.0', '>279.0', 'between 279.0 and 1947.0', '>=1947.0']
['<=8622.5', '<=4078.0', 'between 4078.0 and 8622.5', '>8622.5']
['<=169.5', '<=27.0', 'between 27.0 and 169.5', '>169.5']
['<=71201.5', '<=3.5', 'between 3.5 and 71201.5', '>71201.5', 'between 71201.5

In [3]:
response='ABANDONED'
ct='New'
from feature_clustering import feature_clustering
i='pls'
df_no_pca=dummy_pca
df_no_pca,y, plsca= feature_clustering(df_no_pca,response, i)




sample dataframe shape is:  (100, 519)
{130: 17.86851851851852, 70: 36.554761904761904, 170: 36.187037037037037, 110: 37.644444444444446, 50: 32.786244800950683, 150: 28.498366013071898, 90: 36.02058823529412, 190: 36.631699346405227}
the best # of components is: 110
PLS appraoch


c:\python27\lib\site-packages\sklearn\cross_decomposition\pls_.py:293: UserWarning: Y residual constant at iteration 81
  warnings.warn('Y residual constant at iteration %s' % k)


In [4]:
from clustering import obs_clustering
clust='True'
df=df_no_pca
df, df_w_labels, df_w_clust=obs_clustering(df,response,clust)


(100, 62)
41.8518518519
35.9706959707
36.0071301248
34.5467032967
42.7655677656
43.0036630037
29.2200854701
30
14    65
8     64
19    46
1     46
2     42
28    41
24    38
18    38
20    37
21    36
5     35
26    34
7     33
32    32
29    31
27    30
13    30
3     28
17    26
12    25
33    24
22    21
23    19
10    18
31    18
30    18
15    17
11    16
16    16
4     14
0     14
6     14
25    13
34    13
9      8
Name: LABELS, dtype: int64


In [5]:
from pca_dim_red import pca_code
i='False'
response='ABANDONED'
df_no_pca=df_w_labels
print list(df_no_pca)
df_pca, fitted_pca = pca_code(df_no_pca,response,i)



['LABELS', "AMOUNT_REFUND_pcl_'<=96.0'", u'FILING_STATUS_HeadOfHousehold', u'DMA_AREA_Fort Myers-Naples', u'CHANNEL_FI Channel', "AMOUNT_EITC_pcl_'between 25.0 and 216.5'", 'PRS_SCORE_7.0', 'NUM_W2_5.0', u'DMA_AREA_New York', 'PRS_SCORE_9.0', u'DMA_AREA_Duluth-Superior', u'DMA_AREA_Denver', u'DMA_AREA_San Diego', 'PRS_SCORE_6.0', u'CHANNEL_Paid Search - Brand', u'DMA_AREA_Seattle-Tacoma', "AGE_TAXPAYER_pcl_'between 5.5 and 29.5'", u'IMPORT_TYPE_Partial', u'IMPORT_TYPE_Import', 'PRS_SCORE_10.0', u'DMA_AREA_Houston', u'LAST_STATUS_REJECTED', "AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION_pcl_'>=1527.0'", "TOTAL_REVENUE_pcl_'between 104.98500061035156 and 149.46499633789063'", u'DMA_AREA_Philadelphia', "AGE_SPOUSE_pcl_'between 11.0 and 61.5'", "REJECT_COUNT_pcl_'<=0.5'", 'NUM_SCHC_1.0', u'IMPORT_TYPE_Manual', u'CHANNEL_Organic Search', u'ENTRY_PAGE_GROUP_Product and Pricing Pages', "AMOUNT_TOTAL_INCOME_pcl_'<=617.0'", "AMOUNT_REFUND_pcl_'>112.5'", u'DMA_AREA_Grand Rapids-Kalmazoo-Battle Creek', 

In [ ]:
from get_arrays import get_arrays
i='False'
j='False'
dummy_pca=df_pca
response = 'ABANDONED'
x, y = get_arrays(dummy_pca, response, i, j)

In [ ]:
from model_algorithm import algorithm
response = 'ABANDONED'
models, name = algorithm(x, y, response)

33
Decision_Tree 0.567217514124 0.466101694915 0.46218487395 0.554479418886 117 65 63 55


c:\python27\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Random_Forest 0.572960669274 0.432203389831 0.459459459459 0.570497299311 129 53 67 51
                                                         sig
AMOUNT_TOTAL_INCOME_pcl_'<=617.0'                   0.060768
STATE_ATTACH_COUNT                                  0.054059
PRODUCT_ROLLUP_FREE WEB                             0.046336
CHANNEL_Paid Search - Brand                         0.045146
PRS_SCORE_10.0                                      0.043229
ENTRY_PAGE_GROUP_Homepage                           0.043139
CHANNEL_Organic Search                              0.038476
NUM_W2_3.0                                          0.035013
ENTRY_PAGE_GROUP_Campaign Landing Pages             0.034747
IMPORT_TYPE_Import                                  0.034620
IMPORT_TYPE_Partial                                 0.032489
AGE_TAXPAYER_pcl_'between 5.5 and 29.5'             0.031333
LAST_STATUS_REJECTED                                0.031280
FILING_STATUS_HeadOfHousehold                       0.02984

c:\python27\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


NeuralNetwork 0.546471591515 0.406779661017 0.428571428571 0.544049171168 124 58 70 48
AdaBoost 0.518543432203 0.262711864407 0.340659340659 0.540696591544 149 33 87 31
Naive Bayes 0.682366660548 0.940677966102 0.562025316456 0.514295027007 16 166 7 111
Bernouli Niave Bayes 0.532452282791 0.305084745763 0.375 0.548146768486 144 38 82 36
QDA 0.684576271186 0.949152542373 0.562814070352 0.513037809648 14 168 6 112
Bagging 0.537661644696 0.330508474576 0.391959798995 0.549869621904 140 42 79 39
ERT 0.529868525265 0.389830508475 0.408888888889 0.527332836655 121 61 72 46


c:\python27\lib\site-packages\sklearn\discriminant_analysis.py:695: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


GB 0.550422965338 0.423728813559 0.436681222707 0.544281989197 121 61 68 50
14
Decision_Tree 0.563460683523 0.466101694915 0.458333333333 0.548984913392 115 67 63 55
Random_Forest 0.544303395655 0.381355932203 0.418604651163 0.547820823245 130 52 73 45
                                                         sig
AMOUNT_TOTAL_INCOME_pcl_'<=617.0'                   0.060520
STATE_ATTACH_COUNT                                  0.054315
PRODUCT_ROLLUP_FREE WEB                             0.047091
ENTRY_PAGE_GROUP_Homepage                           0.045080
CHANNEL_Paid Search - Brand                         0.042515
CHANNEL_Organic Search                              0.041042
PRS_SCORE_10.0                                      0.040133
ENTRY_PAGE_GROUP_Campaign Landing Pages             0.036351
IMPORT_TYPE_Import                                  0.034323
IMPORT_TYPE_Partial                                 0.033759
NUM_W2_3.0                                          0.033365
LAST_STATUS_REJ

c:\python27\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


NeuralNetwork 0.580521855486 0.415254237288 0.460093896714 0.58125349227 136 46 69 49
AdaBoost 0.518543432203 0.262711864407 0.340659340659 0.540696591544 149 33 87 31
Naive Bayes 0.682366660548 0.940677966102 0.562025316456 0.514295027007 16 166 7 111
Bernouli Niave Bayes 0.532452282791 0.305084745763 0.375 0.548146768486 144 38 82 36
QDA 0.684576271186 0.949152542373 0.562814070352 0.513037809648 14 168 6 112
Bagging 0.537661644696 0.330508474576 0.391959798995 0.549869621904 140 42 79 39
ERT 0.529868525265 0.389830508475 0.408888888889 0.527332836655 121 61 72 46
GB 0.550422965338 0.423728813559 0.436681222707 0.544281989197 121 61 68 50
13
Decision_Tree 0.558620721856 0.457627118644 0.451882845188 0.544747625256 115 67 64 54
Random_Forest 0.559753916943 0.398305084746 0.437209302326 0.56178990501 132 50 71 47
                                                         sig
AMOUNT_TOTAL_INCOME_pcl_'<=617.0'                   0.062106
STATE_ATTACH_COUNT                                  0

In [ ]:
import pandas as pd
import numpy as np
from predicting_code import transform_to_pca, bin_pca_score_set, get_scoring_arrays, score_set_feature_selection, predict,get_scoring_arrays
defection=pd.DataFrame()
i, j, k=['False','True','False' ]
df_trans_pca = transform_to_pca(scoring_df, fitted_pca, i)
# print df_trans_pca
scoring_df_trans = bin_pca_score_set(df_trans_pca, length_dict, j)

print scoring_df_trans.shape, x.shape
a=list(x)
b=list(scoring_df_trans)
final=list(set(b).difference(a))
scoring_df_trans.drop(final, axis=1, inplace=True)
print scoring_df_trans.shape
x.drop(list(scoring_df_trans), axis=1, inplace=True)
print x.shape
c=list(x)

m=scoring_df_trans.shape[0]     
n= x.shape[1]
print  scoring_df_trans.shape, x.shape

zeros_df=pd.DataFrame(np.zeros((m, n)), columns=c)



scoring_df_trans.reset_index(['CUSTOMER_KEY'], inplace=True)

final_scoring_df=pd.concat([zeros_df,scoring_df_trans ], axis=1)
final_scoring_df.set_index('CUSTOMER_KEY', inplace=True)
print final_scoring_df.shape

scoring_df_trans=final_scoring_df
x_score, index=score_set_feature_selection(scoring_df_trans,plsca, k)
x_score.set_index('CUSTOMER_KEY', inplace=True)
df_p=predict(models,name,x_score, index)
defection=pd.concat([df_p, defection], axis=1)
print defection.head()


In [ ]:
from data_intersection import scoring_data_intersection
data = scoring_data_intersection(df_no_pca, scoring_df)
# print data.head()
print df_cont